In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from torchvision.models import efficientnet_b0
from PIL import Image
from torchvision import transforms
from torch.utils.data import random_split, DataLoader


import albumentations as A
from albumentations.pytorch import ToTensorV2

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
# === 2. AUGMENTATION === #
train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Rotate(limit=30, p=0.5),
    A.Normalize(),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(),
    ToTensorV2(),
])

test_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(),
    ToTensorV2(),
])

In [3]:
class SugarcaneDiseaseDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): Path to the dataset folder (e.g., 'wheat-plant-diseases/train')
            transform (callable, optional): Optional transform to be applied on an image sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_to_idx = {}

        # Get all class names (subdirectory names)
        classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}

        # Collect all image paths and their corresponding labels
        for cls_name in classes:
            cls_folder = os.path.join(root_dir, cls_name)
            if os.path.isdir(cls_folder):
                for img_file in os.listdir(cls_folder):
                    img_path = os.path.join(cls_folder, img_file)
                    self.image_paths.append(img_path)
                    self.labels.append(self.class_to_idx[cls_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        image = np.array(image)  # Albumentations expects NumPy array
    
        if self.transform:
            transformed = self.transform(image=image)
            image = transformed["image"]  # ToTensorV2 returns tensor here
    
        return image, label


In [4]:
# === Load full dataset ===
root_dir = "/kaggle/input/sugarcane-disease-dataset/Sugarcane disease detection" 
full_dataset = SugarcaneDiseaseDataset(root_dir=root_dir, transform=train_transform)

In [5]:
# === Split sizes ===
total_size = len(full_dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

In [6]:
# === Random split ===
train_dataset, val_dataset, test_dataset = random_split(
    full_dataset,
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

In [7]:
val_dataset.dataset.transform = val_transform
test_dataset.dataset.transform = test_transform

In [8]:
    os.cpu_count()

4

In [9]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = len(train_dataset.dataset.class_to_idx)

model = efficientnet_b0(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 130MB/s] 


In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [12]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0

    for images, labels in tqdm(loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()

    return running_loss / len(loader.dataset), correct / len(loader.dataset)

In [13]:
def evaluate(model, loader, criterion):
    model.eval()
    loss = 0.0
    correct = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss += criterion(outputs, labels).item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()

    return loss / len(loader.dataset), correct / len(loader.dataset)

In [15]:
EPOCHS = 10
for epoch in range(EPOCHS):
        print(f"\nEpoch {epoch+1}/{EPOCHS}")
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
        val_loss, val_acc = evaluate(model, val_loader, criterion)

        print(f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f}")
        print(f"Val   Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")


Epoch 1/10


100%|██████████| 491/491 [01:23<00:00,  5.86it/s]


Train Loss: 0.1050, Acc: 0.9618
Val   Loss: 0.1344, Acc: 0.9534

Epoch 2/10


100%|██████████| 491/491 [01:23<00:00,  5.88it/s]


Train Loss: 0.0963, Acc: 0.9667
Val   Loss: 0.1190, Acc: 0.9590

Epoch 3/10


...

Train Loss: 0.0540, Acc: 0.9825
Val   Loss: 0.1223, Acc: 0.9593

Epoch 9/10


100%|██████████| 491/491 [01:24<00:00,  5.80it/s]


Train Loss: 0.0627, Acc: 0.9786
Val   Loss: 0.1460, Acc: 0.9549

Epoch 10/10


100%|██████████| 491/491 [01:24<00:00,  5.81it/s]


Train Loss: 0.0524, Acc: 0.9823
Val   Loss: 0.1751, Acc: 0.9468


In [18]:
model.eval()
loss = 0.0
correct = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss += criterion(outputs, labels).item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()

print(f'loss: {loss / len(test_loader.dataset)}, accuracy: {correct / len(test_loader.dataset)}')

loss: 0.18496798877488166, accuracy: 0.9447743467933492


In [19]:
torch.save(model, 'Sugarcane_model.pth')